In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
#Load the data
data = pd.read_csv('../data/data.csv')
relevent_data = data.drop('code_size', axis=1)
test_app = 'consumer_tiffmedian'

In [4]:
from net import net
losses = []
net = net.to(device)
net.train()

Net(
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv1): Conv1d(1, 10, kernel_size=(5,), stride=(1,), padding=(3,))
  (conv2): Conv1d(10, 20, kernel_size=(5,), stride=(1,), padding=(3,))
  (conv3): Conv1d(20, 10, kernel_size=(5,), stride=(1,), padding=(3,))
  (conv4): Conv1d(10, 1, kernel_size=(5,), stride=(1,), padding=(3,))
  (fc1): Linear(in_features=21, out_features=150, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=150, out_features=200, bias=True)
  (batch_norm): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=200, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=5, bias=True)
)

In [ ]:
def train_test_split(test_app):
  train_data  = relevent_data[relevent_data['APP_NAME'] != test_app]
  test_data = relevent_data[relevent_data['APP_NAME'] == test_app]
  scaler = MinMaxScaler(feature_range=(0,1)).fit(relevent_data[relevent_data['APP_NAME'] != test_app].iloc[:,1:-5])
  scaled_train_predictors = scaler.transform(relevent_data[relevent_data['APP_NAME'] != test_app].iloc[:,1:-5])
  train_targets = relevent_data[relevent_data['APP_NAME'] != test_app].iloc[:,-5:]
  scaled_train_targets = []
  for app in range(0, train_targets.shape[0], 128):
    scaled_train_targets.append(MinMaxScaler(feature_range=(0,1)).fit_transform(train_targets.iloc[app:app+128]))
  scaled_train_targets = np.array(scaled_train_targets).reshape(train_targets.shape[0],5)
  scaled_test_predictors = MinMaxScaler(feature_range=(0,1)).fit_transform(relevent_data[relevent_data['APP_NAME'] == test_app].iloc[:,1:-5])
  scaled_test_targets = MinMaxScaler(feature_range=(0,1)).fit_transform(relevent_data[relevent_data['APP_NAME'] == test_app].iloc[:,-5:])
  train_set = torch.tensor(
      data=np.concatenate(
          (
              scaled_train_predictors,
              scaled_train_targets
          ),
          axis=1
      ),
      dtype=torch.float
      ).to(device)
  test_set = torch.tensor(
    data=np.concatenate(
        (
            scaled_test_predictors,
            scaled_test_targets
        ),
        axis=1
    ),
    dtype=torch.float
  ).to(device)
  return train_set, test_set

In [ ]:
#Set hyperparameters
batch_size = 32
epochs = 50
lr=1e-4
weight_decay = 1e-4

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)

In [ ]:
for test_app in relevent_data['APP_NAME'].unique():
  train_set, test_set = train_test_split(test_app)
  test_losses = []
  #Training loop
  for epoch in range(epochs):
      
      #Set train loss to zero
      running_loss = 0.0
      
      #Shuffle training set
      shuffled_train_set = train_set[torch.randperm(train_set.shape[0])]
      
      for start_index in range(0, shuffled_train_set.shape[0] - batch_size, batch_size):
          #get batches
          X = shuffled_train_set[start_index:start_index + batch_size, :-5].view(batch_size,1,285)
          y = shuffled_train_set[start_index:start_index + batch_size, -5:]
          
          #zero gradients
          optimizer.zero_grad()
          
          #forward pass
          outputs = net(X).view(batch_size,5)
          
          #get loss
          loss = criterion(outputs, y)
          
          #backward pass
          loss.backward()
          optimizer.step()
          running_loss += loss.item()

      #Set test loss to zero
      test_loss = 0.0

      #Shuffle test set
      shuffled_test_set = test_set[torch.randperm(test_set.shape[0])]

      for start_index in range(0, shuffled_test_set.shape[0] - batch_size, batch_size):

          #get batches
          X = shuffled_test_set[start_index:start_index + batch_size, :-5].view(batch_size,1,285)
          y = shuffled_test_set[start_index:start_index + batch_size, -5:]

          #forward pass
          outputs = net(X).view(batch_size,5)

          #get loss
          loss = criterion(outputs, y)
          test_loss += loss.item() * 23
      test_losses.append(test_loss)
    
  print('{}, epoch {}, train loss: {}, test loss: {}, average test_loss {}'.format(test_app, epoch, running_loss, test_loss, sum(test_losses)/len(test_losses) ))


/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


automotive_bitcount, epoch 49, train loss: 5.5290268659591675, test loss: 4.9391521364450455, average test_loss 4.904833242148161


/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [ ]:
from sklearn.metrics import mean_squared_error

from predict import predict
from plot_execution_times import plot_execution_times


predictions = predict(net, 32, test_set)

print('Test Loss:', mean_squared_error(test_set[:,-5:], predictions))
plot_execution_times(predictions, test_set[:, -5:])

